# Layouts

Layout in the context of this package describe the required content of a HDF file. A layout essentially is a HDF itself and is associates with a wrapper class, e.g. `H5File`. The idea behind defining layout is, that the data generation process is supervised. This means, when using `H5File` for filling an HDF5 file a `check()` method will tell, if the minimal required (meta) data has been written. A third user would then only accept the file if a check-run results in zero issues.

The generation and usage of layouts is outlined in the following:

In [ ]:
import h5rdmtoolbox as h5tbx

The already implemented wrapper classes like e.g. `H5File` or `H5Flow` are already associated with layouts: 

In [ ]:
with h5tbx.H5File() as h5:
    lay = h5.layout
    print(lay)

As a `layout` is a HDF file itself, it has similar features like wrapper files, e.g. we can get a nice html representation of the content. Like this we can see what is required to have in any file written with `H5File`:

In [ ]:
lay

## Check layout
Let's create an empty HDF5 file with `h5py`, thus no data will be available. If we then open it with a `H5File` wrapper, everything defined in the layout file is missing, hence, we expect 4 issues to raise:

In [ ]:
import h5py
filename = h5tbx.generate_temporary_filename()
with h5py.File(filename, 'w'):
    pass

with h5tbx.H5File(filename) as h5:
    h5.check()
    print(h5.layout)

## Write an individual layout

Layout file are HDF file, thus we need a filename an then initialize a `Layout` object:

In [ ]:
h5tbx.generate_temporary_filename(suffix='.hdf')
mylayout = h5tbx.conventions.H5Layout(h5tbx.generate_temporary_filename(suffix='.hdf'))
mylayout

Say we expect HDF files to have the following data and structure:
 - a root attribtue "title", which an have any value
 - the datasets "x" and "y" which are 1d-arrays, which must have the attributes "standard_name" or "long_name"
 - a group "meta"

Fill the file with content. Note, the layout file is an HDF file:

In [ ]:
with mylayout.File(mode='w') as h5:
    h5.attrs['title'] = '__any'
    dsx = h5.create_dataset('x', shape=(1,))  # shape does not matter here, but has to be passed
    dsx.attrs['standard_name.alt:long_name'] = '__any'
    dsy = h5.create_dataset('y', shape=(1,))  # shape does not matter here, but has to be passed
    dsy.attrs['standard_name.alt:long_name'] = '__any'
    h5.create_group('meta')
    h5.create_group('meta/other')
mylayout

Now let's create an example HDF5 file and check if the layout is ok or if there are issues:

In [ ]:
test_filename = h5tbx.generate_temporary_filename()
with h5py.File(test_filename, mode='w') as h5:
    h5.attrs['title'] = 'my test file'
    dsx = h5.create_dataset('x', data=[1,2,3])
    dsx.attrs['long_name'] = 'x coordinate'
    dsx.attrs['standard_name'] = 'x_coordinate'
    dsy = h5.create_dataset('y', data=[[1,2],[3,4]])

We use `check()` and pass the opened hdf5 group to check for any issues. We already know, taht the group "meta" and "meta/other" are missing, as well the attribute "standard_name" or "long_name" for dataset "y". Note, that missing group "other" will not raise an issue because "meta" raised one before.

In [ ]:
with h5py.File(test_filename, mode='r') as h5:
    mylayout.check(h5, silent=True)
print(mylayout)

But what exactly is missing? Either change the parameter `silentTrue` to `silent=False` ot run `report()`

In [ ]:
mylayout.report()

Note, that we can enter the check-process at any level. However, no check is performed above that group, in this case the group "meta" which we add before we run a check again beginning from that group level:

In [ ]:
with h5py.File(test_filename, mode='r+') as h5:
    h5.create_group('meta')
    mylayout.check(h5['meta'], silent=True)
print(mylayout)

Only checking the specified group requires to pass `recursive=False`:

In [ ]:
with h5py.File(test_filename, mode='r+') as h5:
    mylayout.check(h5['/'], silent=True, recursive=False)
print(mylayout)